In [1]:
using CairoMakie

import LilGuys as lguys
using Arya
using LilGuys

In [2]:
function sigma_axis(gp; limits=(-1, 2, -2, 2.5), kwargs...) 
	ax = Axis(gp, 
		xlabel=log_r_label,
		ylabel = log_sigma_label,
        limits=limits
		;kwargs...
	)

	return  ax
end

sigma_axis (generic function with 1 method)

In [3]:
function load_profiles(modelname, starsname; extension="stellar_profiles.hdf5")
    path = joinpath(modelsdir, modelname, "stars", starsname, extension)

    parampath = joinpath(modelsdir, modelname, "stars", starsname, "profile.toml")
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        @warn "$parampath not found"
        expected = nothing
    end

    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ], expected
end

load_profiles (generic function with 1 method)

In [4]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-2.5, 1.0, -6, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = sigma_axis(fig[1, 1]; limits=limits, kwargs...)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, prof = profiles[i]
        x = prof.log_r
        y = prof.log_Sigma
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = lguys.calc_Σ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

compare_profiles (generic function with 2 methods)

In [ ]:
function plot_r_percens(profiles; kwargs...)
	fig = Figure()
	ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel="log r / kpc containing stellar mass"; kwargs...)

	q = profiles[1].second.quantiles
	t = [p.second.time for p in profiles] * T2GYR
    
	for i in reverse(eachindex(q))
		Mq = [log10.(p.second.r_quantile[i]) for p in profiles]
        
		lines!(t, Mq, color=q[i], colorrange=extrema(q), label="$(q[i])")
	end

	Legend(fig[1, 2], ax, "quantile")
	fig
end
    

In [7]:
"""
    all_plots(profiles[, expected])

Given the profiles object, this function displayes the plots
for the profiles and r percentiles for each timestep in the profile

"""
function all_plots(profiles, expected=nothing; prof_limits=(-2, 1, -15, 3), title="")
   display(compare_profiles(profiles, expected, 
            sequential=true, legend=false, title=title, limits=prof_limits,
        )
    )

end

all_plots

In [8]:
log_r_label = "log r / kpc"

"log r / kpc"

In [9]:
log_sigma_label = L"$\log \Sigma$ / N\,kpc$^{-2}$"

L"$\log \Sigma$ / N\,kpc$^{-2}$"

In [10]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis/"

"/cosma/home/durham/dc-boye1/data/dwarfs/analysis/"

In [11]:
prof_expected = LilGuys.StellarProfile("/astro/dboyea/dwarfs/observations/ursa_minor/density_profiles/fiducial_profile.toml");

LoadError: UndefVarError: `StellarProfile` not defined in `LilGuys`
Suggestion: check for spelling errors or missing imports.

In [ ]:
import TOML
obs_props = TOML.parsefile("/astro/dboyea/dwarfs/observations/ursa_minor/observed_properties.toml")

In [ ]:
expected_r_kpc = prof_expected.log_r .+ log10(0.0002908882086657216 * obs_props["distance"])

## V=32, r=5.9 (the initial model)

In [ ]:
models = []

In [ ]:
profs, expected = load_profiles("ursa_minor/1e6_v32_r5.0/", "exp2d_rs0.15");


In [ ]:
fig = compare_profiles(profs, expected, sequential=true, legend=false)
errscatter!(expected_r_kpc, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		label="J+24",
		color=:black
	)

fig